In [8]:
import pandas as pd
import numpy as np
import gc

This project uses real-world e-commerce transaction data from Vesta Corporation, a significant payment service provider. The data is split into two files: "identity" and "transaction," with the "TransactionID" column connecting them.

In [38]:
train_transaction = pd.read_csv('datasets_initial/train_transaction.csv',index_col='TransactionID')
train_identity = pd.read_csv('datasets_initial/train_identity.csv',index_col='TransactionID')
test_transaction = pd.read_csv('datasets_initial/test_transaction.csv',index_col='TransactionID')
test_identity = pd.read_csv('datasets_initial/test_identity.csv',index_col='TransactionID')

The identity file does not contain information for all transactions. Both the "identity" and "transaction" files have train and test sets.

In [10]:
# merging datasets
train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_transaction.merge(test_identity,how='left',left_index=True,right_index=True)
del train_transaction, train_identity, test_transaction, test_identity

In [11]:
print(train.shape)
print(test.shape)

(590540, 433)
(506691, 432)


We set a threshold level of 20% for missing values. If a feature has more than 20% of missing values , that feature is dropped.

In [12]:
# exploring nan values
# if a col has: nan >= 20% --> drop

print('Num of columns with missing values more than 20% in train data',(train.isnull().sum() > train.shape[0]*0.2).sum())
print('Num of columns with missing values more than 20% in test data',(test.isnull().sum() > test.shape[0]*0.2).sum())

Num of columns with missing values more than 20% in train data 252
Num of columns with missing values more than 20% in test data 233


In [15]:
# drop features where nan > 20% of total records

def drop_cols_(test,train):
    for col in test.columns:
        if test[col].isnull().sum() >= test.shape[0]*0.2:
                test.drop(col,axis=1,inplace=True)
                if col in train:
                    train.drop(col,axis=1,inplace=True)

In [16]:
drop_cols_(test,train)

In [17]:
train.to_csv('datasets_intermediate/train.csv')
test.to_csv('datasets_intermediate/test.csv')

In [18]:
gc.collect()

169

##### Imputing Missing Values

SimpleImputer - Imputation transformer for completing missing values.

In [19]:
df_train = pd.read_csv('datasets_intermediate/train.csv')
df_test = pd.read_csv('datasets_intermediate/test.csv')

In [20]:
df_test.select_dtypes(include='object')

,ProductCD,card4,card6,P_emaildomain
0,W,visa,debit,gmail.com
1,W,visa,debit,aol.com
2,W,visa,debit,hotmail.com
3,W,visa,debit,gmail.com
4,W,mastercard,debit,gmail.com
...,...,...,...,...
506686,C,mastercard,debit,gmail.com
506687,C,mastercard,debit,hotmail.com
506688,W,visa,debit,hotmail.com
506689,W,mastercard,debit,hotmail.com


In [21]:
df_test.select_dtypes(exclude='object').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506691 entries, 0 to 506690
Columns: 196 entries, TransactionID to V321
dtypes: float64(193), int64(3)
memory usage: 757.7 MB


In [23]:
# identifying columns with nan values
def id_nan(df):
    df_cols = []
    for col in df.columns:
        if df[col].isnull().any() == True:
            df_cols.append(col)
    return df_cols

In [24]:
# identified which columns have nan values
n_train_list = id_nan(df_train)
n_test_list = id_nan(df_test)

In [25]:
df_train.select_dtypes(include='object').isnull().any()

ProductCD        False
card4             True
card6             True
P_emaildomain     True
id_12             True
id_15             True
id_16             True
id_23             True
id_27             True
id_28             True
id_29             True
id_30             True
id_31             True
id_33             True
id_34             True
id_35             True
id_36             True
id_37             True
id_38             True
dtype: bool

In [26]:
# imputer
def imputer(df):
    import numpy as np
    from sklearn.impute import SimpleImputer
    mean = SimpleImputer(missing_values=np.nan,strategy='mean')
    mode = SimpleImputer(missing_values=np.nan,strategy='most_frequent')
    for col in df.columns:
        if df[col].isnull().any()==True:
            if df[col].dtypes=='O':
                df[col]=mode.fit_transform(df[[col]])
            else:
                df[col]=mean.fit_transform(df[[col]])      
    return df

In [27]:
imputer(df_train)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38
0,2987000,0,86400,68.50,W,13926,362.555488,150.0,discover,142.0,...,Found,Windows 10,chrome 63.0,26.508597,1920x1080,match_status:2,T,F,T,F
1,2987001,0,86401,29.00,W,2755,404.000000,150.0,mastercard,102.0,...,Found,Windows 10,chrome 63.0,26.508597,1920x1080,match_status:2,T,F,T,F
2,2987002,0,86469,59.00,W,4663,490.000000,150.0,visa,166.0,...,Found,Windows 10,chrome 63.0,26.508597,1920x1080,match_status:2,T,F,T,F
3,2987003,0,86499,50.00,W,18132,567.000000,150.0,mastercard,117.0,...,Found,Windows 10,chrome 63.0,26.508597,1920x1080,match_status:2,T,F,T,F
4,2987004,0,86506,50.00,H,4497,514.000000,150.0,mastercard,102.0,...,NotFound,Android 7.0,samsung browser 6.2,32.000000,2220x1080,match_status:2,T,F,T,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0,15811047,49.00,W,6550,362.555488,150.0,visa,226.0,...,Found,Windows 10,chrome 63.0,26.508597,1920x1080,match_status:2,T,F,T,F
590536,3577536,0,15811049,39.50,W,10444,225.000000,150.0,mastercard,224.0,...,Found,Windows 10,chrome 63.0,26.508597,1920x1080,match_status:2,T,F,T,F
590537,3577537,0,15811079,30.95,W,12037,595.000000,150.0,mastercard,224.0,...,Found,Windows 10,chrome 63.0,26.508597,1920x1080,match_status:2,T,F,T,F
590538,3577538,0,15811088,117.00,W,7826,481.000000,150.0,mastercard,224.0,...,Found,Windows 10,chrome 63.0,26.508597,1920x1080,match_status:2,T,F,T,F


In [28]:
df_train.isnull().any()

TransactionID     False
isFraud           False
TransactionDT     False
TransactionAmt    False
ProductCD         False
                  ...  
id_34             False
id_35             False
id_36             False
id_37             False
id_38             False
Length: 239, dtype: bool

In [29]:
imputer(df_test)

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321
0,3663549,18403224,31.950,W,10409,111.0,150.0,visa,226.0,debit,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,3663550,18403263,49.000,W,4272,111.0,150.0,visa,226.0,debit,...,77.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,3663551,18403310,171.000,W,4476,574.0,150.0,visa,226.0,debit,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,263.0,0.0
3,3663552,18403310,284.950,W,10989,360.0,150.0,visa,166.0,debit,...,0.000000,282.540009,282.540009,282.540009,0.0,0.0,0.0,0.0,0.0,0.0
4,3663553,18403317,67.950,W,18018,452.0,150.0,mastercard,117.0,debit,...,67.949997,67.949997,183.850006,67.949997,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506686,4170235,34214279,94.679,C,13832,375.0,185.0,mastercard,224.0,debit,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
506687,4170236,34214287,12.173,C,3154,408.0,185.0,mastercard,224.0,debit,...,0.000000,31.723700,31.723700,31.723700,0.0,0.0,0.0,0.0,0.0,0.0
506688,4170237,34214326,49.000,W,16661,490.0,150.0,visa,226.0,debit,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
506689,4170238,34214337,202.000,W,16621,516.0,150.0,mastercard,224.0,debit,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
df_test.isnull().any()

TransactionID     False
TransactionDT     False
TransactionAmt    False
ProductCD         False
card1             False
                  ...  
V317              False
V318              False
V319              False
V320              False
V321              False
Length: 200, dtype: bool

In [31]:
# saved to new csv file to avoid re-imputing incase of errors in future
df_train.to_csv('datasets_intermediate/imputed_train.csv')
df_test.to_csv('datasets_intermediate/imputed_test.csv')

In [32]:
train = pd.read_csv('datasets_intermediate/imputed_train.csv')
test  = pd.read_csv('datasets_intermediate/imputed_test.csv')

In [33]:
# some email like gmail.com is written in different ways (gmail.com and gmail)
train['P_emaildomain'].unique()

array(['gmail.com', 'outlook.com', 'yahoo.com', 'mail.com',
       'anonymous.com', 'hotmail.com', 'verizon.net', 'aol.com', 'me.com',
       'comcast.net', 'optonline.net', 'cox.net', 'charter.net',
       'rocketmail.com', 'prodigy.net.mx', 'embarqmail.com', 'icloud.com',
       'live.com.mx', 'gmail', 'live.com', 'att.net', 'juno.com',
       'ymail.com', 'sbcglobal.net', 'bellsouth.net', 'msn.com', 'q.com',
       'yahoo.com.mx', 'centurylink.net', 'servicios-ta.com',
       'earthlink.net', 'hotmail.es', 'cfl.rr.com', 'roadrunner.com',
       'netzero.net', 'gmx.de', 'suddenlink.net', 'frontiernet.net',
       'windstream.net', 'frontier.com', 'outlook.es', 'mac.com',
       'netzero.com', 'aim.com', 'web.de', 'twc.com', 'cableone.net',
       'yahoo.fr', 'yahoo.de', 'yahoo.es', 'sc.rr.com', 'ptd.net',
       'live.fr', 'yahoo.co.uk', 'hotmail.fr', 'hotmail.de',
       'hotmail.co.uk', 'protonmail.com', 'yahoo.co.jp'], dtype=object)

In [34]:
# renaming gmail to gmail.com
def update_cell(df):
    for i in df.index:
        if df.at[i,'P_emaildomain']=='gmail':
            df.at[i,'P_emaildomain']='gmail.com'
    return df

In [35]:
update_cell(train)

,Unnamed: 0,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,...,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38
0,0,2987000,0,86400,68.50,W,13926,362.555488,150.0,discover,...,Found,Windows 10,chrome 63.0,26.508597,1920x1080,match_status:2,T,F,T,F
1,1,2987001,0,86401,29.00,W,2755,404.000000,150.0,mastercard,...,Found,Windows 10,chrome 63.0,26.508597,1920x1080,match_status:2,T,F,T,F
2,2,2987002,0,86469,59.00,W,4663,490.000000,150.0,visa,...,Found,Windows 10,chrome 63.0,26.508597,1920x1080,match_status:2,T,F,T,F
3,3,2987003,0,86499,50.00,W,18132,567.000000,150.0,mastercard,...,Found,Windows 10,chrome 63.0,26.508597,1920x1080,match_status:2,T,F,T,F
4,4,2987004,0,86506,50.00,H,4497,514.000000,150.0,mastercard,...,NotFound,Android 7.0,samsung browser 6.2,32.000000,2220x1080,match_status:2,T,F,T,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,590535,3577535,0,15811047,49.00,W,6550,362.555488,150.0,visa,...,Found,Windows 10,chrome 63.0,26.508597,1920x1080,match_status:2,T,F,T,F
590536,590536,3577536,0,15811049,39.50,W,10444,225.000000,150.0,mastercard,...,Found,Windows 10,chrome 63.0,26.508597,1920x1080,match_status:2,T,F,T,F
590537,590537,3577537,0,15811079,30.95,W,12037,595.000000,150.0,mastercard,...,Found,Windows 10,chrome 63.0,26.508597,1920x1080,match_status:2,T,F,T,F
590538,590538,3577538,0,15811088,117.00,W,7826,481.000000,150.0,mastercard,...,Found,Windows 10,chrome 63.0,26.508597,1920x1080,match_status:2,T,F,T,F


In [36]:
update_cell(test)

,Unnamed: 0,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321
0,0,3663549,18403224,31.950,W,10409,111.0,150.0,visa,226.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,1,3663550,18403263,49.000,W,4272,111.0,150.0,visa,226.0,...,77.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,2,3663551,18403310,171.000,W,4476,574.0,150.0,visa,226.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,263.0,0.0
3,3,3663552,18403310,284.950,W,10989,360.0,150.0,visa,166.0,...,0.000000,282.540009,282.540009,282.540009,0.0,0.0,0.0,0.0,0.0,0.0
4,4,3663553,18403317,67.950,W,18018,452.0,150.0,mastercard,117.0,...,67.949997,67.949997,183.850006,67.949997,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506686,506686,4170235,34214279,94.679,C,13832,375.0,185.0,mastercard,224.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
506687,506687,4170236,34214287,12.173,C,3154,408.0,185.0,mastercard,224.0,...,0.000000,31.723700,31.723700,31.723700,0.0,0.0,0.0,0.0,0.0,0.0
506688,506688,4170237,34214326,49.000,W,16661,490.0,150.0,visa,226.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
506689,506689,4170238,34214337,202.000,W,16621,516.0,150.0,mastercard,224.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
# # save as csv for data_exploration further
train.to_csv('datasets_cleaned/final_train.csv')
test.to_csv('datasets_cleaned/final_test.csv')